In [1]:
push!(LOAD_PATH, ".")
using BenchmarkTools
using IntervalSets

In [2]:
# Knots
struct Knots
    vector :: Array{Float64,1}
    function Knots(vector)
        new(sort(vector))
    end
end

Base.:+(k₁::Knots, k₂::Knots) = Knots(sort([k₁.vector...,k₂.vector...]))
Base.:*(p₊::Int, k::Knots) = if (p₊==0)
        Knots([])
    elseif (p₊>0)
        sum(k for _ ∈ 1:p₊)
    else
        error("p₊ must be non-negative")
    end
    
Base.in(r::Real, k::Knots) = in(r,k.vector)
Base.getindex(k::Knots, i::Int) = k.vector[i]
Base.getindex(k::Knots, v::AbstractArray{Int64,1}) = Knots(k.vector[v])
Base.length(k::Knots) = length(k.vector)
♯(k::Knots) = length(k::Knots)
Base.firstindex(k) = 1
Base.lastindex(k) = length(k)
Base.unique(k::Knots) = Knots(unique(k.vector))

function Base.:⊆(k::Knots, k′::Knots)
    K′=copy(k′.vector)
    for kᵢ ∈ k.vector
        i=findfirst(x->x==kᵢ,K′)
        if(i isa Nothing)
            return false
        end
        deleteat!(K′,i)
    end
    return true
end

In [3]:
# B-Spline Space
struct BSplineSpace
    degree::Int
    knots::Knots
    function BSplineSpace(degree::Int, knots::Knots)
        if (degree < 0)
            error("degree of polynominal must be non-negative")
        else
            new(degree,knots)
        end
    end
end

const 𝒫 = BSplineSpace

function dim(bsplinespace::BSplineSpace)
    p=bsplinespace.degree
    k=bsplinespace.knots
    return length(k)-p-1
end

function Base.:⊆(P::BSplineSpace, P′::BSplineSpace)
    p=P.degree
    k=P.knots
    p′=P′.degree
    k′=P′.knots
    p₊=p′-p
    
    return (k+p₊*unique(k) ⊆ k′) && p₊ ≥ 0
end

function Base.iszero(P::BSplineSpace)
    p=P.degree
    k=P.knots
    n=dim(P)
    return [k[i]==k[i+p+1] for i ∈ 1:n]
end

In [4]:
# B-Spline functions
function BSplineBasis₊₀(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots
    
    n=length(k)-p-1
    if(p==0)
        @inbounds return [k[i] ≤ t < k[i+1] for i ∈ 1:n]
        # @inbounds return [k[i] ≤ t < k[i+1] || (k[i] ≠ k[i+1] == k[end] == t) for i ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,(t-k[i])/(k[i+p]-k[i])) for i ∈ 1:n+1]
        @inbounds B=BSplineBasis₊₀(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]+(1-K[i+1])*B[i+1] for i ∈ 1:n]
    end
end

function BSplineBasis₋₀(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots
    
    n=length(k)-p-1
    if(p==0)
        @inbounds return [k[i] < t ≤ k[i+1] for i ∈ 1:n]
        # @inbounds return [k[i] < t ≤ k[i+1] || (k[i] ≠ k[i+1] == k[1] == t) for i ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,(t-k[i])/(k[i+p]-k[i])) for i ∈ 1:n+1]
        @inbounds B=BSplineBasis₋₀(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]+(1-K[i+1])*B[i+1] for i ∈ 1:n]
    end
end

BSplineBasis = BSplineBasis₊₀

function BSplineBasis′₊₀(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots

    n=length(k)-p-1
    if(p==0)
        return [0.0 for _ ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,p/(k[i+p]-k[i])) for i ∈ 1:n+1]
        B=BSplineBasis₊₀(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]-K[i+1]*B[i+1] for i ∈ 1:n]
    end
end

function BSplineBasis′₋₀(P::BSplineSpace, t)::Array{Float64,1}
    p=P.degree
    k=P.knots

    n=length(k)-p-1
    if(p==0)
        return [0.0 for _ ∈ 1:n]
    else
        @inbounds K=[ifelse(k[i+p]==k[i],0,p/(k[i+p]-k[i])) for i ∈ 1:n+1]
        B=BSplineBasis₋₀(𝒫(p-1,k),t)
        @inbounds return [K[i]*B[i]-K[i+1]*B[i+1] for i ∈ 1:n]
    end
end

BSplineBasis′ = BSplineBasis₊₀′

function BSplineSupport(i::Int64, P::BSplineSpace)::ClosedInterval
    p=P.degree
    k=P.knots

    return k[i]..k[i+p+1]
end

function BSplineCoefficient(P::BSplineSpace, P′::BSplineSpace)::Array{Float64,2}
    p=P.degree
    k=P.knots
    p′=P′.degree
    k′=P′.knots

    p₊=p′-p
    if(P ⊆ P′)
        if(p == 0)
            n=length(k)-1
            n′=length(k′)-p₊-1
            A⁰=Float64[BSplineSupport(j,𝒫(p₊,k′)) ⊆ BSplineSupport(i,𝒫(0,k)) for i ∈ 1:n, j ∈ 1:n′]
            A⁰[:,findall(iszero(P′))].=NaN
            return A⁰
        else
            Aᵖ⁻¹=BSplineCoefficient(𝒫(p-1, k), 𝒫(p′-1, k′))
            n=dim(P)
            n′=dim(P′)
            
            Z=iszero(𝒫(p′-1,k′))
            W=findall(Z)

            K′=[k′[i+p′]-k′[i] for i ∈ 1:n′+1]
            K=[ifelse(k[i+p]≠k[i], 1/(k[i+p]-k[i]), 0.0) for i ∈ 1:n+1]
            global Δ=(p/p′)*[K′[j]*(K[i]*Aᵖ⁻¹[i,j]-K[i+1]*Aᵖ⁻¹[i+1,j]) for i ∈ 1:n, j ∈ 1:n′+1]
            Aᵖ=zeros(n,n′)
            Aᵖ[:,1]=Δ[:,1]
            Aᵖ[:,n′]=-Δ[:,n′+1]
            
            if(length(W)==0)
                Q=[1:n′]
            else
                Q=[1:W[1]-1,[W[i]:W[i+1]-1 for i ∈ 1:length(W)-1]...,W[end]:n′]
            end
            l=length(Q)
            L=length.(Q)
            Ãᵖ=[Aᵖ[:,q] for q ∈ Q]
            
            for ȷ ∈ 2:l-1
                if(L[ȷ]==1)
                    Ãᵖ[ȷ] .= NaN
                end
            end
            for ȷ ∈ 1:l-1
                if(L[ȷ] ≥ 2)
                    t=k′[W[ȷ]]
                    Ãᵖ[ȷ][:,end]=BSplineBasis₋₀(𝒫(p,k),t)
                end
            end
            for ȷ ∈ 2:l
                if(L[ȷ] ≥ 2)
                    t=k′[W[ȷ-1]+p]
                    Ãᵖ[ȷ][:,1]=BSplineBasis₊₀(𝒫(p,k),t)
                end
            end
            for ȷ ∈ 1:l
                if(L[ȷ] ≥ 3)
                    r=Q[ȷ]
                    A₊=copy(Ãᵖ[ȷ])
                    A₋=copy(Ãᵖ[ȷ])
                    for j ∈ 1:L[ȷ]-2
                        A₊[:,j+1]=A₊[:,j]+Δ[:,j+r[1]]
                        A₋[:,L[ȷ]-j]=A₋[:,L[ȷ]-j+1]-Δ[:,L[ȷ]-j+r[1]]
                    end
                    Ãᵖ[ȷ]=(A₊+A₋)/2
                end
            end
            
            Aᵖ=hcat(Ãᵖ...)
            return Aᵖ .* Float64[BSplineSupport(j,P′) ⊆ BSplineSupport(i,P) for i ∈ 1:n, j ∈ 1:n′]
        end
    else
        error("𝒫[p,k] ⊄ 𝒫[p′,k′]")
    end
end

BSplineCoefficient (generic function with 1 method)

In [26]:
p=3
p′=50
p₊=p′-p
k=Knots([0,0,0,0])+Knots(2*rand(3))+Knots([2,2,2,2])+Knots([1.1,1.1,1.1,1.1])
k′=k+(p₊)*unique(k)+Knots(rand(4).*(k[end]-k[1]).+k[1])

P=𝒫(p,k)
P′=𝒫(p′,k′)

BSplineSpace(50, Knots([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]))

In [27]:
t=2*rand()
BSplineBasis(𝒫(p,k),t)-BSplineCoefficient(𝒫(p,k),𝒫(p′,k′))*BSplineBasis(𝒫(p′,k′),t)

11-element Array{Float64,1}:
  0.0                   
  1.0547118733938987e-15
  1.2212453270876722e-15
  6.38378239159465e-16  
 -2.688821387764051e-16 
  0.0                   
  0.0                   
  0.0                   
  0.0                   
  0.0                   
  0.0                   

In [44]:
function ⊗(X::Array{Float64},Y::Array{Float64})::Array{Float64}
    m=size(X)
    n=size(Y)
    reshape(reshape(X,length(X)) * reshape(Y,length(Y))', m..., n...)
end

function tensorprod(X::Array{T,1}) where T <: Array{Float64}
    n=length(X)
    # X[1] ⊗ … ⊗ X[n]
    @inbounds Y=X[1]
    for i ∈ 2:n
        @inbounds Y = Y ⊗ X[i]
    end
    return Y
end

tensorprod (generic function with 1 method)

In [30]:
struct BSplineManifold
    bsplinespaces::Array{BSplineSpace,1}
    controlpoints::Array{Float64}
    function BSplineManifold(bsplinespaces::Array{BSplineSpace,1}, controlpoints::Array{Float64})
        if (collect(size(controlpoints)[1:end-1]) ≠ dim.(bsplinespaces))
            error("dimension does not match")
        else
            new(bsplinespaces, controlpoints)
        end
    end
end

function BSplineBasis(𝒫s::Array{BSplineSpace,1},t)
    if(length(𝒫s)==length(t)==1)
        return BSplineBasis(𝒫s[1],t[1])
    elseif(length(𝒫s)==length(t)==2)
        return BSplineBasis(𝒫s[1],t[1])*BSplineBasis(𝒫s[2],t[2])'
    else
        error("dimension does not match")
    end
end

function Mapping(M::BSplineManifold, t::Array{Float64,1})
    𝒫s = M.bsplinespaces
    𝒂 = M.controlpoints
    d=length(𝒫s)
    d̂=size(𝒂)[end]
    return [sum(BSplineBasis(𝒫s,t).*𝒂[:,:,i]) for i ∈ 1:d̂]
end

Mapping (generic function with 1 method)

In [45]:
function BSplineBasis(𝒫s::Array{BSplineSpace,1},t)
    d=length(t)
    Bs=[BSplineBasis(𝒫s[i],t[i]) for i ∈ 1:d]
    return tensorprod(Bs)
end

BSplineBasis (generic function with 2 methods)

In [43]:
[(i) for (i,j) ∈ (1:8, 3:90)]

2-element Array{Int64,1}:
 1
 3

In [38]:
[i for i ⊆ [1,2,4]]

LoadError: syntax: invalid iteration specification

In [46]:
function Refinement(M::BSplineManifold, 𝒫s′::Array{BSplineSpace,1})
    𝒫s = M.bsplinespaces
    𝒂 = M.controlpoints
    d̂ = size(𝒂)[end]
    n = dim.(𝒫s)
    n′ = dim.(𝒫s′)
    if(prod(𝒫s .⊆ 𝒫s′))
        A = BSplineCoefficient.(𝒫s,𝒫s′)
        𝒂′ = [sum(A[1][I₁,J₁]*A[2][I₂,J₂]*𝒂[I₁,I₂,i] for I₁ ∈ 1:n[1], I₂ ∈ 1:n[2]) for J₁ ∈ 1:n′[1], J₂ ∈ 1:n′[2], i ∈ 1:d̂]
        return BSplineManifold(𝒫s′, 𝒂′)
    else
        error("𝒫[p,k] ⊄ 𝒫[p′,k′]")
    end
end

Refinement (generic function with 1 method)

In [55]:
Knots([1,2,3]) ⊈ Knots([1,2,3])

false

In [58]:
?⊈

"⊈" can be typed by \nsubseteq<tab>

search: ⊈



```
⊈(a, b) -> Bool
⊉(b, a) -> Bool
```

Negation of `⊆` and `⊇`, i.e. checks that `a` is not a subset of `b`.

# Examples

```jldoctest
julia> (1, 2) ⊈ (2, 3)
true

julia> (1, 2) ⊈ (1, 2, 3)
false
```


In [59]:
@less ⊈([1,2],[2,3])

# This file is a part of Julia. License is MIT: https://julialang.org/license

eltype(::Type{<:AbstractSet{T}}) where {T} = @isdefined(T) ? T : Any
sizehint!(s::AbstractSet, n) = nothing

copy!(dst::AbstractSet, src::AbstractSet) = union!(empty!(dst), src)

## set operations (union, intersection, symmetric difference)

"""
    union(s, itrs...)
    ∪(s, itrs...)

Construct the union of sets. Maintain order with arrays.

# Examples
```jldoctest
julia> union([1, 2], [3, 4])
4-element Array{Int64,1}:
 1
 2
 3
 4

julia> union([1, 2], [2, 4])
3-element Array{Int64,1}:
 1
 2
 4

julia> union([4, 2], 1:2)
3-element Array{Int64,1}:
 4
 2
 1

julia> union(Set([1, 2]), 2:3)
Set([2, 3, 1])
```
"""
function union end

_in(itr) = x -> x in itr

union(s, sets...) = union!(emptymutable(s, promote_eltype(s, sets...)), s, sets...)
union(s::AbstractSet) = copy(s)

const ∪ = union

"""
    union!(s::Union{AbstractSet,AbstractVector}, itrs...)

Construct the union of passed in sets and overwrite `s` with

In [51]:
⊄

UndefVarError: UndefVarError: ⊄ not defined

In [47]:
k₁=k₂=Knots([-0.1,0,1,1.1])
p₁=p₂=1

𝒂=Float64[ifelse(i==1, 2*I₁, 3*I₂) for I₁ ∈ 0:1,  I₂ ∈ 0:1, i ∈ 1:2]
𝒫s=[𝒫(p₁,k₁), 𝒫(p₂,k₂)]
M=BSplineManifold(𝒫s,𝒂)

t=[0.2,0.3]
Mapping(M,t)

2-element Array{Float64,1}:
 0.39999999999999997
 0.8999999999999999 

In [48]:
p′₁=p′₂=3
k′₁=k₁+2unique(k₁)+Knots(rand(300))
k′₂=k₂+2unique(k₂)+Knots(rand(300))
M′=Refinement(M, [𝒫(p′₁,k′₁), 𝒫(p′₂,k′₂)])

BSplineManifold(BSplineSpace[BSplineSpace(3, Knots([-0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.005139389503538094, 0.008904766112976636, 0.01225753306945876, 0.015636762284422234  …  0.9912895675478157, 0.9943770426597411, 0.9949696421537422, 0.9960480681084651, 1.0, 1.0, 1.0, 1.1, 1.1, 1.1])), BSplineSpace(3, Knots([-0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.001873070211117156, 0.00215299582708961, 0.00420073372994878, 0.010156852030194008  …  0.9945144778033306, 0.9967341930586378, 0.9993673657017237, 0.9999345968753, 1.0, 1.0, 1.0, 1.1, 1.1, 1.1]))], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.4444444444444445 0.8888888888888888 … 0.8888888888888888 0.4444444444444445; 0.2222222222222222 0.4444444444444443 … 0.4444444444444443 0.2222222222222222]

[0.0 0.0 … 0.6666666666666665 0.3333333333333333; 0.0 0.0 … 1.3333333333333333 0.6666666666666667; … ; 0.0 0.0 … 1.3333333333333333 0.6666666666666667; 0.0 0.0 … 0.6666666666666665 0.3333333333333333])

In [50]:
@benchmark BSplineBasis(𝒫s,[0.3,0.5])

2×2 Array{Float64,2}:
 0.35  0.35
 0.15  0.15

In [11]:
t=rand(2)
@benchmark Mapping(M′,t)

BenchmarkTools.Trial: 
  memory estimate:  4.66 MiB
  allocs estimate:  14979
  --------------
  minimum time:     645.087 μs (0.00% GC)
  median time:      723.404 μs (0.00% GC)
  mean time:        891.210 μs (12.82% GC)
  maximum time:     4.528 ms (31.89% GC)
  --------------
  samples:          5562
  evals/sample:     1

In [66]:
@benchmark Mapping(M,t)

BenchmarkTools.Trial: 
  memory estimate:  5.00 KiB
  allocs estimate:  121
  --------------
  minimum time:     5.430 μs (0.00% GC)
  median time:      5.808 μs (0.00% GC)
  mean time:        6.537 μs (8.41% GC)
  maximum time:     946.530 μs (98.35% GC)
  --------------
  samples:          10000
  evals/sample:     6

In [13]:
p=4
p′=17
p₊=p′-p
k=Knots((9*rand(20)).+3)
k₊=Knots(((k[end]-k[1])*rand(13)).+k[1])
k′=k+p₊*unique(k)+k₊

Knots([3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397, 3.3196085066852397  …  11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171, 11.976017802056171])

In [14]:
@benchmark BSplineBasis(𝒫(p,k),6.3)  # ちょっと遅いのはノット列とかを毎度型検査(?)してるから?

BenchmarkTools.Trial: 
  memory estimate:  3.03 KiB
  allocs estimate:  41
  --------------
  minimum time:     2.005 μs (0.00% GC)
  median time:      2.148 μs (0.00% GC)
  mean time:        2.508 μs (8.17% GC)
  maximum time:     509.071 μs (98.40% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [15]:
BSplineSpace(3,Knots([1,2,3]))

BSplineSpace(3, Knots([1.0, 2.0, 3.0]))

In [16]:
A=BSplineCoefficient(𝒫(p,k), 𝒫(p′,k′))

15×275 Array{Float64,2}:
  1.81166e-5   0.000160061   0.000701802  …  -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
 -0.0         -0.0          -0.0          …   0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0          …  -0.0          -0.0       
 -0.0         -0.0          -0.0              0.0           0.0       
  0.0          0.0           0.0             -0.0          -0.0       
  0.0          0.0           0.0             -0.0   

In [17]:
𝒫(p,k) ⊆ 𝒫(p′,k′)

true

In [21]:
1.0 ∈ Knots([1])

true

In [20]:
Base.in(r::Real, k::Knots) = in(r,k.vector)

In [22]:
missing

missing

In [24]:
null

UndefVarError: UndefVarError: null not defined

In [23]:
Araryaundef

array initializer with undefined values

In [32]:
hoge=Array{Float64}(undef, 3,3)

3×3 Array{Float64,2}:
 3.8382e151   4.50622e-144  1.41529e161
 1.47722e179  3.80985e180   6.00737e-67
 8.37404e242  1.06396e224   3.88664e97 

In [34]:
hoge[1,1]=nothing

MethodError: MethodError: Cannot `convert` an object of type Nothing to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [31]:
nothing

In [36]:
?empty

search: empty empty! isempty



```
empty(x::Tuple)
```

Returns an empty tuple, `()`.

---

```
empty(v::AbstractVector, [eltype])
```

Create an empty vector similar to `v`, optionally changing the `eltype`.

# Examples

```jldoctest
julia> empty([1.0, 2.0, 3.0])
0-element Array{Float64,1}

julia> empty([1.0, 2.0, 3.0], String)
0-element Array{String,1}
```

---

```
empty(a::AbstractDict, [index_type=keytype(a)], [value_type=valtype(a)])
```

Create an empty `AbstractDict` container which can accept indices of type `index_type` and values of type `value_type`. The second and third arguments are optional and default to the input's `keytype` and `valtype`, respectively. (If only one of the two types is specified, it is assumed to be the `value_type`, and the `index_type` we default to `keytype(a)`).

Custom `AbstractDict` subtypes may choose which specific dictionary type is best suited to return for the given index and value types, by specializing on the three-argument signature. The default is to return an empty `Dict`.


In [39]:
EmptySet

UndefVarError: UndefVarError: EmptySet not defined

In [38]:
emptyset

UndefVarError: UndefVarError: emptyset not defined

In [41]:
1 ∈ Set([1,2,3])

true

In [43]:
Set([1,2,3,1])

Set([2, 3, 1])

In [1]:
Set([1,2,3])

Set([2, 3, 1])